In [52]:
!pip install tqdm

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

# 데이터 로드
data = pd.read_excel('../dataset/VMI_data.xls')
data['CRET_TIME'] = pd.to_datetime(data['CRET_TIME'], format='%Y%m%d%H%M')

# 투입예정 수량
columns_to_keep = [col for col in data.columns if "투입예정 수량" in col or "투입계획" in col]
columns_to_keep.append("Part Number")
columns_to_keep.append("CRET_TIME")
data_selected = data[columns_to_keep]

# 피봇테이블 생성
pivot_data = data_selected.pivot_table(index="Part Number", columns="CRET_TIME", values=columns_to_keep[:-2], aggfunc="sum")
pivot_data_filled = pivot_data.fillna(0)

# X와 y로 분리
X = pivot_data_filled.values[:, :-1]
y = pivot_data_filled.values[:, -2]

# 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=100)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.9, random_state=100)

# 스케일링 적용
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)


# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X = torch.tensor(X_data, dtype=torch.float32)
        self.y = torch.tensor(y_data, dtype=torch.float32).view(-1, 1) # 회귀에 맞게 shape 변경
        self.len = len(X_data)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
        
    def __len__(self):
        return self.len

# DataLoader 생성
BATCH_SIZE = 3
train_data = CustomDataset(X_train, y_train)
val_data = CustomDataset(X_val, y_val)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

# 모델 정의
class RegressionModel(nn.Module):
    def __init__(self, num_features):
        super(RegressionModel, self).__init__()
        self.layer_1 = nn.Linear(num_features, 128)
        self.layer_2 = nn.Linear(128, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, 1) # 회귀 출력은 1개
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.relu(x)
        x = self.layer_2(x)
        x = self.relu(x)
        x = self.layer_3(x)
        x = self.relu(x)
        x = self.layer_out(x)
        return x

# 모델 인스턴스화
NUM_FEATURES = X_train.shape[1]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RegressionModel(NUM_FEATURES)
model.to(device)

# 손실 함수와 옵티마이저
criterion = nn.MSELoss() # 회귀에 사용
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 학습 진행
EPOCHS = 50
for epoch in tqdm(range(EPOCHS)):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

    # 성능 지표 계산
    with torch.no_grad():
        model.eval()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            train_loss += loss.item()
        
        train_rmse = torch.sqrt(torch.mean((y_pred - y_batch) ** 2))
        train_mae = torch.mean(torch.abs(y_pred - y_batch))
        total_variance = torch.sum((y_batch - torch.mean(y_batch)) ** 2)
        explained_variance = torch.sum((y_pred - torch.mean(y_batch)) ** 2)
        train_r2 = 1 - (explained_variance / total_variance)

        print(f'Epoch {epoch}: | Train Loss: {train_loss/len(train_loader):.3f} \
        | Train RMSE: {train_rmse.item():.3f} | Train MAE: {train_mae.item():.3f} | Train R^2: {train_r2.item():.3f}')

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 0: | Train Loss: 51.303         | Train RMSE: 2.608 | Train MAE: 2.608 | Train R^2: -inf
Epoch 1: | Train Loss: 323.847         | Train RMSE: 0.913 | Train MAE: 0.913 | Train R^2: -inf
Epoch 2: | Train Loss: 22.991         | Train RMSE: 0.114 | Train MAE: 0.114 | Train R^2: -inf
Epoch 3: | Train Loss: 100.776         | Train RMSE: 0.926 | Train MAE: 0.926 | Train R^2: -inf
Epoch 4: | Train Loss: 21.239         | Train RMSE: 0.573 | Train MAE: 0.573 | Train R^2: -inf
Epoch 5: | Train Loss: 33.475         | Train RMSE: 0.369 | Train MAE: 0.369 | Train R^2: -inf
Epoch 6: | Train Loss: 100.460         | Train RMSE: 14.222 | Train MAE: 14.222 | Train R^2: -inf
Epoch 7: | Train Loss: 38.737         | Train RMSE: 15.952 | Train MAE: 15.952 | Train R^2: -inf
Epoch 8: | Train Loss: 19.870         | Train RMSE: 0.593 | Train MAE: 0.593 | Train R^2: -inf
Epoch 9: | Train Loss: 19.404         | Train RMSE: 0.695 | Train MAE: 0.695 | Train R^2: -inf
Epoch 10: | Train Loss: 18.830         | Tr

In [68]:
# 테스트 데이터에 대한 평가
model.eval()
test_loss = 0
test_rmse = 0
test_mae = 0
test_r2_score = 0

with torch.no_grad():
    y_test_all = torch.tensor([], dtype=torch.float32).to(device)
    y_pred_all = torch.tensor([], dtype=torch.float32).to(device)

    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch)
        test_loss += loss.item()

        y_test_all = torch.cat((y_test_all, y_batch), 0)
        y_pred_all = torch.cat((y_pred_all, y_pred), 0)

    test_rmse = torch.sqrt(torch.mean((y_pred_all - y_test_all) ** 2))
    test_mae = torch.mean(torch.abs(y_pred_all - y_test_all))

    total_variance = torch.sum((y_test_all - torch.mean(y_test_all)) ** 2)
    explained_variance = torch.sum((y_pred_all - y_test_all) ** 2)
    test_r2_score = 1 - (explained_variance / total_variance)

# 평균 성능 지표 출력
test_loss /= len(test_loader)

print(f'Test Loss: {test_loss:.3f} | Test RMSE: {test_rmse:.3f} | Test MAE: {test_mae:.3f} | Test R^2: {test_r2_score:.3f}')

Test Loss: 5.545 | Test RMSE: 3.031 | Test MAE: 1.750 | Test R^2: 0.725


In [58]:
pivot_data

D+10일 투입예정 수량                                          \
CRET_TIME   2021-09-13 18:30:00 2021-09-14 06:05:00 2021-09-14 06:25:00   
Part Number                                                               
Part 0                      0.0                 0.0                 0.0   
Part 1                      0.0                 0.0                 0.0   
Part 10                    73.0                73.0                32.0   
Part 100                    0.0                 0.0                 0.0   
Part 101                    0.0                 0.0                 0.0   
...                         ...                 ...                 ...   
Part 95                   167.0               167.0               277.0   
Part 96                     0.0                 0.0                 0.0   
Part 97                     0.0                 0.0                 0.0   
Part 98                     0.0                 0.0                 0.0   
Part 99                     0.0                 0.0                 0.0   

                                                                         \
CRET_TIME   2021-09-14 06:34:00 2021-09-14 17:30:00 2021-09-15 06:20:00   
Part Number                                                               
Part 0                      0.0                 0.0                 0.0   
Part 1                      0.0                 0.0                 0.0   
Part 10                    32.0                32.0                 0.0   
Part 100                    0.0                 0.0                 0.0   
Part 101                    0.0                 0.0                 0.0   
...                         ...                 ...                 ...   
Part 95                   277.0               277.0               249.0   
Part 96                     0.0                 0.0                 NaN   
Part 97                     0.0                 0.0                 0.0   
Part 98                     0.0                 0.0                 0.0   
Part 99                     0.0                 0.0                 0.0   

                                                                         \
CRET_TIME   2021-09-15 06:37:00 2021-09-15 16:33:00 2021-09-16 06:32:00   
Part Number                                                               
Part 0                      0.0                 0.0                 0.0   
Part 1                      0.0                 0.0                 0.0   
Part 10                     0.0                 0.0                 0.0   
Part 100                    0.0                 0.0                 0.0   
Part 101                    0.0                 0.0                 0.0   
...                         ...                 ...                 ...   
Part 95                   249.0               249.0                 0.0   
Part 96                     NaN                 NaN                 0.0   
Part 97                     0.0                 0.0                 0.0   
Part 98                     0.0                 0.0                 0.0   
Part 99                     0.0                 0.0                 0.0   

                                 ... D일23~01(02)H 투입계획(발주) 수량  \
CRET_TIME   2021-09-16 06:38:00  ...      2021-10-28 15:30:00   
Part Number                      ...                            
Part 0                      0.0  ...                      0.0   
Part 1                      0.0  ...                      0.0   
Part 10                     0.0  ...                      8.0   
Part 100                    0.0  ...                      0.0   
Part 101                    0.0  ...                      NaN   
...                         ...  ...                      ...   
Part 95                     0.0  ...                     24.0   
Part 96                     0.0  ...                      0.0   
Part 97                     0.0  ...                      0.0   
Part 98                     0.0  ...                      0.0   
Part 99             